# 4. Pipelines
- how to use pipelines to clean up your modeling code.

## 설명
- pipelines의 장점
  1. Cleaner code: 직접 training, validation data를 추적할 필요가 없다.
  2. Fewer bugs: 전처리 과정에서 필요한 step을 빼먹을 가능성이 낮아진다.
  3. Easier to Productionize: prototype에서 활용가능한 모델로 transition하는 데 도움을 준다.
  4. More Options for Model Validation: cross-validation 같은 다양한 모델 validation 옵션들을 사용할 수 있다.

<hr>

- libraries and modules
  - ColumnTransformer: 서로 다른 전처리 과정을 한 번에 처리해주는 클래스
    1. numeric data: 결측치 처리
    2. categorical data: one-hot encoding
  - Pipeline: Pipeline 클래스를 활용하여 전처리와 모델링 과정을 합친 pipeline을 정의할 수 있다.
    - training data 전처리와 모델 fitting까지 한 번에 실행된다.
    - validation data의 전처리를 자동으로 해준다.

## Exercise

In [66]:
# load dataset
import pandas as pd

df_train = pd.read_csv('data/housing_prices_train.csv', index_col='Id')
df_test = pd.read_csv('data/housing_prices_test.csv', index_col='Id')

In [67]:
# make data

from sklearn.model_selection import train_test_split

# target 칼럼에서 결측값이 있는 행 삭제
df_train.dropna(subset=['SalePrice'], axis=0, inplace=True)

# X_full, y 정의
y = df_train.SalePrice
X_full = df_train.drop(columns=['SalePrice'], axis=1)

# category columns, numeric columns 정의
categorical_cols = [
    col for col in X_full.columns
    if X_full[col].dtype == 'object' and 
    X_full[col].nunique() < 10  # 고유값이 10개 이상인 칼럼은 사용하지 않음
]
numeric_cols = [
    col for col in X_full.columns
    if X_full[col].dtype in ['int64', 'float64']
]
# numeric_data = X_train.select_dtypes(
#     include=['int64', 'float64']
# )

# train, valid data 분리
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
    X_full,
    y,
    train_size=0.8,
    test_size=0.2,
    random_state=0
)

# 모델 build 및 평가에 사용될 데이터 정의
use_cols = categorical_cols + numeric_cols
X_train = X_train_full[use_cols].copy()
X_valid = X_valid_full[use_cols].copy()
X_test = df_test[use_cols].copy()

In [68]:
# preprocessing

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# for numeric data
numeric_transformer = SimpleImputer(
    strategy='mean'
)

# for categorical data
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

# 각 type에 따른 transformer를 종합하는 preprocessor 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [70]:
# build and test a model

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Pipeline으로 transformer와 모델을 합치기
model = RandomForestRegressor(random_state=0, n_estimators=100)
pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ]
)

# 전처리 및 fitting 진행
pipeline.fit(X_train, y_train)

# validation data 전처리 및 예측
preds = pipeline.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))


MAE: 17648.417157534244
